In [1]:
import sys
import os
import findspark
findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable




In [2]:
import spark
import pyspark
#from secret.txt import user,password

In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *



In [4]:
#creating a spark session
spark = SparkSession.builder.appName('capstone_project').getOrCreate()


In [5]:
#read the database username and password from secret.txt
secrets_file = os.path.join("files", "secret.txt")
with open(secrets_file, "r") as f:		
    lines = f.readlines()
for line in lines:
    words = line.split("=")
    if (words[0] == "user"):
        user = words[1].strip()
    elif (words[0] == "password"):
        password = words[1].strip()


In [6]:
cdw_sapp_branch = os.path.join("files", "cdw_sapp_branch.json")

df_branch = spark.read.json(cdw_sapp_branch)


In [7]:

type(df_branch)
df_branch.show()

+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|         Paters

In [8]:
df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [9]:
    
df_branch.createOrReplaceTempView('branch_data')

In [10]:


spark.sql("SELECT BRANCH_CODE, BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, BRANCH_STATE, \
IFNULL(branch_zip, 00000) BRANCH_ZIP, CONCAT('(', SUBSTRING(branch_phone,1,3),')', SUBSTRING(branch_phone, 4,3), '-', SUBSTRING(branch_phone,7)) BRANCH_PHONE, LAST_UPDATED \
FROM branch_data")


DataFrame[BRANCH_CODE: bigint, BRANCH_NAME: string, BRANCH_STREET: string, BRANCH_CITY: string, BRANCH_STATE: string, BRANCH_ZIP: bigint, BRANCH_PHONE: string, LAST_UPDATED: string]

In [11]:
branch_df = spark.sql("SELECT CAST(BRANCH_CODE AS INT) BRANCH_CODE, BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, BRANCH_STATE, \
IFNULL(branch_zip, 00000) BRANCH_ZIP, CONCAT('(', SUBSTRING(branch_phone,1,3),')', SUBSTRING(branch_phone, 4,3), '-', SUBSTRING(branch_phone,7)) BRANCH_PHONE, LAST_UPDATED \
FROM branch_data")
branch_df.show()

+-----------+------------+-------------------+-----------------+------------+----------+-------------+--------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|        LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+----------+-------------+--------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18T16:51:...|
|          2|Example Bank|  Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18T16:51:...|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18T16:51:...|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|     32068|(123)466-3064|2018-04-18T16:51:...|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|     19406|(123)484-9701|2018-04-18T16:51:...|
|       

In [12]:
branch_df.printSchema()

root
 |-- BRANCH_CODE: integer (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = false)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [13]:
#sc.sql("SELECT CAST(BRANCH_CODE AS INT) BRANCH_CODE, CAST(BRANCH_ZIP as INT) BRANCH_ZIP FROM branch_data")
branch_df.createOrReplaceTempView('branch_df')

In [14]:
branch_df = spark.sql("SELECT BRANCH_CODE, BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, BRANCH_STATE, \
CAST(BRANCH_ZIP as INT) BRANCH_ZIP,BRANCH_PHONE , LAST_UPDATED \
FROM branch_df")


In [15]:
branch_df.printSchema()

root
 |-- BRANCH_CODE: integer (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_ZIP: integer (nullable = false)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [16]:
branch_df.show()

+-----------+------------+-------------------+-----------------+------------+----------+-------------+--------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|        LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+----------+-------------+--------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18T16:51:...|
|          2|Example Bank|  Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18T16:51:...|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18T16:51:...|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|     32068|(123)466-3064|2018-04-18T16:51:...|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|     19406|(123)484-9701|2018-04-18T16:51:...|
|       

In [17]:
df_branch = branch_df.withColumn('LAST_UPDATED', F.to_timestamp('LAST_UPDATED'))

In [18]:
df_branch.printSchema()

root
 |-- BRANCH_CODE: integer (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_ZIP: integer (nullable = false)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)



In [19]:
df_branch.show()

+-----------+------------+-------------------+-----------------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+----------+-------------+-------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18 15:51:47|
|          2|Example Bank|  Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18 15:51:47|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18 15:51:47|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|     32068|(123)466-3064|2018-04-18 15:51:47|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|     19406|(123)484-9701|2018-04-18 15:51:47|
|          7|Exa

In [20]:
df_branch.printSchema()

root
 |-- BRANCH_CODE: integer (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_ZIP: integer (nullable = false)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)



In [21]:
df_branch.show()

+-----------+------------+-------------------+-----------------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+----------+-------------+-------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18 15:51:47|
|          2|Example Bank|  Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18 15:51:47|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18 15:51:47|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|     32068|(123)466-3064|2018-04-18 15:51:47|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|     19406|(123)484-9701|2018-04-18 15:51:47|
|          7|Exa

In [22]:

df_branch.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.cdw_sapp_branch") \
.option("user", user) \
.option("password", password) \
.save()


In [23]:
spark.stop()

In [24]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_branch` 
# CHANGE COLUMN `BRANCH_NAME` `BRANCH_NAME` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_STREET` `BRANCH_STREET` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_CITY` `BRANCH_CITY` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_STATE` `BRANCH_STATE` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_PHONE` `BRANCH_PHONE` VARCHAR(45) NULL DEFAULT NULL ;
